<a href="https://colab.research.google.com/github/ddv246/DSBA-Project/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Predicting New York City Yellow Taxi Demand**

## **Business Problem:**

---
In the New York city, people use taxi at a much higher frequency than most places. Instead
of booking customers by phone ahead of time, there is still a majority of New York taxi
drivers that pick up passengers on street. However, as per recent studies, Taxi patronage
has considerably declined since 2011 due to competition from
rideshare
services like Uber,
Lyft etc. The ability to predict taxi ridership could present valuable insights to such taxi
dispatchers that would in turn help them in making important decisions that could revive
their profit margin - such as how to position cabs where they are most needed, how many
taxis to dispatch, and how ridership varies over time. Our project focuses on **predicting the
number of taxi pickups** given a one-hour time window and a location within New York City.


# **Use Case Scenario:**

---

For any given location in New York City, our goal is to predict the number of pickups in that
given location at particular time interval. Some location require more taxis at a particular
time than other locations due to their proximity to offices, schools, hospitals etc. With the
competition that the Yellow Taxi drivers are facing due to the advent of apps like Uber and
Lyft, we propose a solution that will help them predict the taxi demand in a particular area.
The prediction result can be transferred to the taxi drivers via Mobile app’s, and they can
subsequently move to the locations where the predicted pickups are higher and so they can
profit more than they normally would.


# **Data Information:**

---

**Source of Data:** https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page 

Here, we have used October 2015 data.

**Information on Taxi:**

**Yellow Taxi: Yellow Medallion Taxicabs**

Trips made by New York City’s iconic yellow taxis have been recorded and provided to the TLC
since 2009. Yellow taxis are traditionally hailed by signaling to a driver who is on duty and seeking
a passenger (street hail). The pickups are not pre-arranged.
Yellow taxis are the only vehicles permitted to respond to a street hail from a passenger in all five
boroughs. 


In [8]:
#If opening in colab run this cell
!git clone https://github.com/ddv246/DSBA-Project
%cd DSBA-Project/YellowTaxi

Cloning into 'DSBA-Project'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 51 (delta 23), reused 23 (delta 5), pack-reused 0
Unpacking objects: 100% (51/51), done.
/content/DSBA-Project/YellowTaxi/DSBA-Project/YellowTaxi


In [0]:

import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium

from datetime import datetime
import time
import seaborn as sns
import os
import math
import xgboost as xgb
import matplotlib
matplotlib.use('nbagg')
#from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.cluster import MiniBatchKMeans, KMeans
import warnings
warnings.simplefilter('ignore')


In [0]:
#import glob


#all_files1 = []
#for name in glob.glob('YellowTaxi/*'):
 #   all_files1.append(name)

#li = []


#for filename in all_files1:
 #   df = pd.read_csv(filename, index_col=None, header=0)
  #  li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)



In [10]:
df=pd.read_csv("2015_sample.csv")
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,1,N,-73.944351,40.754578,2,6.00,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2015-10-01 00:00:00,2015-10-01 00:00:00,1,7.68,-73.989937,40.743439,1,N,-73.986687,40.689129,2,27.50,0.0,0.5,0.00,0.0,0.3,28.30
2,2,2015-10-01 00:00:00,2015-10-01 00:00:00,2,2.53,-73.987328,40.720020,1,N,-73.999084,40.744381,1,12.50,0.0,0.5,2.00,0.0,0.3,15.30
3,2,2015-10-01 00:00:00,2015-10-01 00:00:00,0,1.20,-73.953758,40.743385,5,N,-73.930008,40.736622,2,25.26,0.0,0.5,0.00,0.0,0.3,26.06
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,1,N,-73.959869,40.801323,1,15.50,0.5,0.5,3.00,0.0,0.3,19.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,1,N,-74.002640,40.728294,1,20.50,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,1,N,-73.997917,40.720467,1,9.00,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,1,N,-73.991631,40.754829,1,16.00,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,1,N,-73.964859,40.769749,1,13.00,0.0,0.5,2.76,0.0,0.3,16.56


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorID               99999 non-null  int64  
 1   tpep_pickup_datetime   99999 non-null  object 
 2   tpep_dropoff_datetime  99999 non-null  object 
 3   passenger_count        99999 non-null  int64  
 4   trip_distance          99999 non-null  float64
 5   pickup_longitude       99999 non-null  float64
 6   pickup_latitude        99999 non-null  float64
 7   RatecodeID             99999 non-null  int64  
 8   store_and_fwd_flag     99999 non-null  object 
 9   dropoff_longitude      99999 non-null  float64
 10  dropoff_latitude       99999 non-null  float64
 11  payment_type           99999 non-null  int64  
 12  fare_amount            99999 non-null  float64
 13  extra                  99999 non-null  float64
 14  mta_tax                99999 non-null  float64
 15  ti

# **Features in Dataset:** 

---
Field Name | 	Description
--- | ---
**VendorID** | A code indicating the TPEP provider that provided the record. 
             | 1= Creative Mobile Technologies, LLC; 2= VeriFone Inc
**tpep_pickup_datetime** | The date and time when the meter was engaged. 
**tpep_dropoff_datetime**| The date and time when the meter was disengaged. 
**Passenger_count** | The number of passengers in the vehicle. This is a driver-entered value
**Trip_distance**  | The elapsed trip distance in miles reported by the taximeter.
**PULocationID** | TLC Taxi Zone in which the taximeter was engaged
**DOLocationID** | TLC Taxi Zone in which the taximeter was disengaged
**RateCodeID** | The final rate code in effect at the end of the trip.
**Store_and_fwd_flag** | This flag indicates whether the trip record was held in  vehicle memory before sending to the vendor,
| because the vehicle did not have a connection to the server.
**Payment_type** | A numeric code signifying how the passenger paid for the trip. 
**Fare_amount** | The time-and-distance fare calculated by the meter.
**Extra** | Miscellaneous extras and surcharges. Currently, this only includes
      |the $0.50  and  $1 rush hour and overnight charges. 
**Improvement_surcharge** |$0.30 improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015.
**Tip_amount** |Tip amount – This field is automatically populated for credit card tips.Cash tips are not included.
**Tolls_amount** |Total amount of all tolls paid in trip.
**Total_amount** |The total amount charged to passengers. Does not include cash tips.
**MTA_Tax** | 50 cent MTA tax that is automatically triggered based on the metered rate in use.




## **DATA CLEANING**
As a process of Data Cleaning and univariate analysis of the data, all errorneous and outlier points will be removed. Some of them listed below,



1.   **Checking Uniqueness of Vendor_ID.**
2.   **Checking if drop off time is earlier than pick up time.**

1.   **Checking if passenger_count is less than 0.**
2.   **Checking if Trip distance is less than 0.1 miles.**

1.   **Checking if Payment type is invalid.**
2.   **Checking if Latitute and Longtitude fall within NYC.**







In [12]:
df.isnull().sum()


VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
RatecodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

#### **Dropping NA**

In [0]:
df=df.dropna()


#### **Deleting unnecessary columns**

In [0]:
del df['RatecodeID']

In [0]:
del df ['store_and_fwd_flag']

In [0]:
df.drop(df[df.trip_distance ==0.00].index, inplace=True)

In [17]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,-73.944351,40.754578,2,6.00,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2015-10-01 00:00:00,2015-10-01 00:00:00,1,7.68,-73.989937,40.743439,-73.986687,40.689129,2,27.50,0.0,0.5,0.00,0.0,0.3,28.30
2,2,2015-10-01 00:00:00,2015-10-01 00:00:00,2,2.53,-73.987328,40.720020,-73.999084,40.744381,1,12.50,0.0,0.5,2.00,0.0,0.3,15.30
3,2,2015-10-01 00:00:00,2015-10-01 00:00:00,0,1.20,-73.953758,40.743385,-73.930008,40.736622,2,25.26,0.0,0.5,0.00,0.0,0.3,26.06
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,-73.959869,40.801323,1,15.50,0.5,0.5,3.00,0.0,0.3,19.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,-74.002640,40.728294,1,20.50,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,-73.997917,40.720467,1,9.00,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,-73.991631,40.754829,1,16.00,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,-73.964859,40.769749,1,13.00,0.0,0.5,2.76,0.0,0.3,16.56


### **1. Checking uniqueness of ID**

In [18]:
print("Number of ids in the train dataset: ", len(df["VendorID"]))
print("Number of unique ids in the train dataset: ", len(pd.unique(df["VendorID"])), "\n")

Number of ids in the train dataset:  99174
Number of unique ids in the train dataset:  2 



**OBSERVATION:**

Vendor_ID takes on only two values , 1 or 2. If the hypothesis is right and the values in the Vendor_ID column represent the data from two different taxi companies. 

### **2.Checking if drop off time is earlier than pick up time**

In [19]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
(df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).describe()

count                     99174
mean     0 days 00:15:18.138715
std      0 days 00:47:51.121904
min           -1 days +23:58:44
25%             0 days 00:06:21
50%             0 days 00:10:43
75%             0 days 00:17:34
max             0 days 23:59:17
dtype: object

#### **Removing rows where dropoff time is earlier than pickup time**

In [20]:
import numpy as np

df=df[(df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]) /np.timedelta64(1, 'D') > 0.0]
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,-73.944351,40.754578,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,-73.959869,40.801323,1,15.5,0.5,0.5,3.00,0.0,0.3,19.80
5,1,2015-10-01 00:00:01,2015-10-01 00:13:41,1,3.10,-73.975296,40.751396,-73.970924,40.785984,1,12.5,0.5,0.5,1.00,0.0,0.3,14.80
6,1,2015-10-01 00:00:01,2015-10-01 00:21:23,1,4.50,-73.997078,40.722290,-73.960472,40.761517,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80
7,2,2015-10-01 00:00:01,2015-10-01 00:05:50,1,0.84,-73.998711,40.734756,-74.003082,40.742546,1,6.0,0.5,0.5,0.70,0.0,0.3,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,-74.002640,40.728294,1,20.5,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,-73.997917,40.720467,1,9.0,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,-73.991631,40.754829,1,16.0,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,-73.964859,40.769749,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56


### **3.Checking if passenger_count is less than 0**

In [21]:
df.passenger_count.describe()

count    99161.000000
mean         1.613195
std          1.321662
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: passenger_count, dtype: float64

In [22]:
df=df[df.passenger_count >= 1]
df.passenger_count.describe()

count    99160.000000
mean         1.613211
std          1.321659
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: passenger_count, dtype: float64

#### **OBSERVATION:**
A trip cannot happen without passenger, therefore, removing rows with passenger_count < 1

### **4. Checking if Trip distance is less than 0.1 miles.**

In [23]:
df.trip_distance.describe()

count    99160.000000
mean         3.098804
std          3.825100
min          0.010000
25%          1.020000
50%          1.800000
75%          3.410000
max        225.800000
Name: trip_distance, dtype: float64

In [24]:
df[df.trip_distance==0.1]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
912,1,2015-10-14 10:25:52,2015-10-14 10:33:07,1,0.1,-73.979515,40.743816,-73.977898,40.746399,1,6.0,0.0,0.5,0.00,0.00,0.3,6.80
1358,1,2015-10-14 10:27:19,2015-10-14 10:31:45,1,0.1,-73.976395,40.752182,-73.976395,40.752182,1,4.5,0.0,0.5,1.32,0.00,0.3,6.62
1684,1,2015-10-14 10:28:24,2015-10-14 10:29:10,1,0.1,-73.976013,40.791401,-73.975189,40.790958,2,2.5,0.0,0.5,0.00,0.00,0.3,3.30
1985,1,2015-10-14 10:29:27,2015-10-14 11:06:22,1,0.1,-73.888039,40.767563,-73.973091,40.757507,2,2.5,0.0,0.5,0.00,5.54,0.3,8.84
2434,1,2015-10-14 10:30:54,2015-10-14 10:32:25,4,0.1,-73.978043,40.752914,-73.977478,40.751820,1,3.0,0.0,0.5,0.75,0.00,0.3,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95590,1,2015-10-01 09:19:49,2015-10-01 09:25:31,1,0.1,-73.951530,40.791168,-73.953491,40.791164,2,5.0,0.0,0.5,0.00,0.00,0.3,5.80
96738,2,2015-10-01 09:23:11,2015-10-01 09:23:58,1,0.1,-74.004303,40.707561,-74.002876,40.708431,2,2.5,0.0,0.5,0.00,0.00,0.3,3.30
97043,1,2015-10-01 09:24:01,2015-10-01 09:25:13,1,0.1,-73.976715,40.759525,-73.977608,40.758144,2,3.0,0.0,0.5,0.00,0.00,0.3,3.80
97280,1,2015-10-01 09:24:44,2015-10-01 09:28:56,1,0.1,-73.977715,40.764503,-73.974709,40.763504,1,4.5,0.0,0.5,1.05,0.00,0.3,6.35


In [25]:
df=df[df.trip_distance >= 0.1]
df.trip_distance.describe()

count    99021.000000
mean         3.103100
std          3.826064
min          0.100000
25%          1.030000
50%          1.800000
75%          3.420000
max        225.800000
Name: trip_distance, dtype: float64

**OBSERVATION:**
Trip distance cannot be less than 0.1 miles ( 0.1 miles as considering a passenger might hops in to a taxi but hops off due to traffic). Thus removing all such rows 

### **5.Checking if Payment type is invalid.**

In [26]:
df.payment_type.describe()

count    99021.000000
mean         1.328163
std          0.481902
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          4.000000
Name: payment_type, dtype: float64

**Description:**

Payment types: 

A numeric code signifying how the passenger paid for the trip.

1= Credit card

2= Cash

3= No charge

4= Dispute

### **6. Checking if Latitute and Longtitude fall within NYC** 

### **PICKUP:**



In [27]:
#detecting the pickups latitude and longitudes which are outside NYC.
out_nyc=df[((df.pickup_latitude <= 40.5774) | (df.pickup_longitude <= -74.15) | (df.pickup_latitude >= 40.9176) | (df.pickup_longitude >= -73.7004))]
out_nyc

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
15,1,2015-10-01 00:00:03,2015-10-01 00:03:57,1,0.60,0.0,0.0,0.0,0.0,1,4.5,0.5,0.5,1.16,0.0,0.3,6.96
185,2,2015-10-14 10:23:18,2015-10-14 10:32:23,1,1.33,0.0,0.0,0.0,0.0,1,8.0,0.0,0.5,1.76,0.0,0.3,10.56
265,1,2015-10-14 10:23:36,2015-10-14 10:32:39,1,0.90,0.0,0.0,0.0,0.0,1,7.0,0.0,0.5,1.55,0.0,0.3,9.35
358,2,2015-10-14 10:23:56,2015-10-14 10:39:58,1,2.66,0.0,0.0,0.0,0.0,2,12.5,0.0,0.5,0.00,0.0,0.3,13.30
456,2,2015-10-14 10:24:18,2015-10-14 10:26:49,1,0.18,0.0,0.0,0.0,0.0,2,3.5,0.0,0.5,0.00,0.0,0.3,4.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99707,1,2015-10-01 09:32:16,2015-10-01 09:43:02,1,2.00,0.0,0.0,0.0,0.0,1,9.5,0.0,0.5,2.05,0.0,0.3,12.35
99713,2,2015-10-01 09:32:16,2015-10-01 09:41:48,1,0.88,0.0,0.0,0.0,0.0,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
99715,2,2015-10-01 09:32:16,2015-10-01 09:47:31,1,1.60,0.0,0.0,0.0,0.0,1,11.0,0.0,0.5,2.95,0.0,0.3,14.75
99799,2,2015-10-01 09:32:29,2015-10-01 09:46:00,1,0.91,0.0,0.0,0.0,0.0,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80


#### **The Pickup Map - Before Cleaning:**

In [28]:
m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")

outside_pickups = out_nyc.head(25000)

for i,j in outside_pickups.iterrows():
    if j["pickup_latitude"] != 0:
        folium.Marker([j["pickup_latitude"], j["pickup_longitude"]]).add_to(m)
m

### **The Pickup Map - After Cleaning:**

In [0]:
df = df[(((df.pickup_latitude >= 40.5774) & (df.pickup_latitude <= 40.9176)) & ((df.pickup_longitude >= -74.15) & (df.pickup_longitude <= -73.7004)))]




In [30]:

m = folium.Map(location = [40.9176, -73.7004], tiles = "Stamen Toner")

pickups_within_NYC = df.sample(n = 500)

for i,j in pickups_within_NYC.iterrows():
    folium.Marker([j["pickup_latitude"], j["pickup_longitude"]]).add_to(m)
m

**OBSERVATION:** 

Now we can see that all the  pick ups are concentrated in and around NYC.

### **DROPOFF:**

In [31]:
out_nyc = df[((df.dropoff_latitude <= 40.5774) | (df.dropoff_longitude <= -74.15) | (df.dropoff_latitude >= 40.9176) | (df.dropoff_longitude >= -73.7004))]
out_nyc



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
472,1,2015-10-14 10:24:23,2015-10-14 10:55:25,2,17.00,-73.980553,40.765854,-74.182304,40.687759,1,65.5,0.0,0.0,16.00,14.25,0.3,96.05
524,2,2015-10-14 10:24:33,2015-10-14 11:05:26,2,18.10,-73.989029,40.736687,-74.183281,40.688072,1,72.5,0.0,0.0,17.53,14.85,0.3,105.18
628,2,2015-10-14 10:24:55,2015-10-14 11:05:54,1,25.30,-73.875664,40.773701,-73.767532,41.032471,1,98.5,0.0,0.5,10.00,5.54,0.3,114.84
1316,2,2015-10-14 10:27:11,2015-10-14 11:02:21,3,16.59,-73.984032,40.757648,-74.183510,40.688187,1,66.5,0.0,0.0,16.82,17.29,0.3,100.91
1825,1,2015-10-14 10:28:54,2015-10-14 10:58:57,1,8.70,-73.963593,40.774342,0.000000,0.000000,1,31.0,0.0,0.5,3.00,0.00,0.3,34.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97324,2,2015-10-01 09:24:52,2015-10-01 09:49:36,6,14.92,-74.005737,40.725399,-74.177017,40.691490,2,58.0,0.0,0.0,0.00,14.00,0.3,72.30
98586,2,2015-10-01 09:28:47,2015-10-01 10:04:02,1,16.28,-73.988815,40.744560,-74.177689,40.690327,2,65.0,0.0,0.0,0.00,19.00,0.3,84.30
98814,1,2015-10-01 09:29:30,2015-10-01 09:50:56,1,1.40,-73.986900,40.736492,0.000000,0.000000,1,13.5,0.0,0.5,2.85,0.00,0.3,17.15
99692,1,2015-10-01 09:32:13,2015-10-01 10:20:32,1,3.60,-73.969872,40.799454,0.000000,0.000000,1,28.0,0.0,0.5,10.00,0.00,0.3,38.80


#### **The Dropoff Map - Before cleaning:**

In [32]:
m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")

outside_dropoff = out_nyc.head(25000)

for i,j in outside_dropoff.iterrows():
    if j["dropoff_latitude"] != 0:
        folium.Marker([j["dropoff_latitude"], j["dropoff_longitude"]]).add_to(m)
m

**OBSERVATION:**

As you can see in the above map, there are many erroneous data points. Some of which are in other state or outside of NYC.  All these erroneous data points are removed. 

In [0]:
df = df[(((df.dropoff_latitude >= 40.5774) & (df.dropoff_latitude <= 40.9176)) & ((df.dropoff_longitude >= -74.15) & (df.dropoff_longitude <= -73.7004)))]


### **The Dropoff Map - After Cleaning:**

In [34]:

m = folium.Map(location = [40.9176, -73.7004], tiles = "Stamen Toner")

dropoff_within_NYC = df.sample(n = 500)

for i,j in dropoff_within_NYC.iterrows():
    folium.Marker([j["dropoff_latitude"], j["dropoff_longitude"]]).add_to(m)
m

**OBSERVATION:** 

Now we can see that all the drop offs are concentrated in and around NYC.